In [1]:
import xarray as xr
import numpy as np
from osgeo import gdal
from pathlib import Path
import gc
# rows, cols = 1934, 2242 #we will need to calculate this, see the other notebook
# months = 2 # will be 12

_vars = ['tmin', 'tmax']


from temds.datasources import worldclim

# from temds import subprograms
# gdal.UseExceptions() ## gdal 4.0 future proofing
import os

In [2]:
out_srs = 'EPSG:6931'
extent_file = '/media/rwspicer/data/V3/tem/00-aoi/aoi-5km-buffer-6931.tiff'
from importlib import reload
import rioxarray  # activate the rio accessor
reload(worldclim)

months = list(range(1,13))
print(f'months {months}')
# wc_test = worldclim.WorldClim(months, '', True)
# wc_test.new_from_raster_extent(extent_file)

months [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [3]:
# config = 'my-config.yml'
# wc_test = subprograms.download('worldclim', config=config)


wc = worldclim.WorldClim(
    '/media/rwspicer/data/V3/tem/01-unprocessed/worldclim/', 
    extent_file,
    verbose = True,
    cleanup_uncompressed = False
)


extent (-4602000.0, 4251000.0, 4366000.0, -3485000.0)
resolution, 4000.0,-4000.0
out size 2242, 1934
setting tmin from raw data at '/media/rwspicer/data/V3/tem/01-unprocessed/worldclim'
loading tmin data from /media/rwspicer/data/V3/tem/01-unprocessed/worldclim/tmin/wc2.1_30s_tmin_01.tif at index 0
.. Running gdal.Warp to extent (-4602000.0, -3481000.0, 4366000.0, 4255000.0)
loading tmin data from /media/rwspicer/data/V3/tem/01-unprocessed/worldclim/tmin/wc2.1_30s_tmin_02.tif at index 1
.. Running gdal.Warp to extent (-4602000.0, -3481000.0, 4366000.0, 4255000.0)
loading tmin data from /media/rwspicer/data/V3/tem/01-unprocessed/worldclim/tmin/wc2.1_30s_tmin_03.tif at index 2
.. Running gdal.Warp to extent (-4602000.0, -3481000.0, 4366000.0, 4255000.0)
loading tmin data from /media/rwspicer/data/V3/tem/01-unprocessed/worldclim/tmin/wc2.1_30s_tmin_04.tif at index 3
.. Running gdal.Warp to extent (-4602000.0, -3481000.0, 4366000.0, 4255000.0)
loading tmin data from /media/rwspicer/data/V3

In [4]:
print(wc.save.__doc__)

Save `dataset` as a netCDF file.

        Parameters
        ----------
        out_file: path
            file to save
        missing_value: float, default 1.e+20
        fill_value: float, default 1.e+20
            values set as _FillValuem, and missing_value in netCDF variable
            headers
        


In [5]:
wc.save('/media/rwspicer/data/V3/tem/02-arctic/worldclim/worldclim-arctic.nc')

TypeError: an integer is required

In [23]:
self = wc
out_file = '/media/rwspicer/data/V3/tem/02-arctic/worldclim/worldclim-arctic.nc'
kwargs={'overwrite': True}

lookup = lambda kw, ke, de: kw[ke] if ke in kw else de

fill_value = lookup(kwargs, 'fill_value', 1.0e+20 )
missing_value = lookup(kwargs, 'missing_value', 1.0e+20 )
compress = lookup(kwargs, 'use_zlib', True)
complevel = lookup(kwargs, 'complevel', 9 )
overwrite = lookup(kwargs, 'overwrite', False )

if 'climate_encoding' in kwargs:            
    climate_enc = kwargs['climate_encoding']
else:
    climate_enc = {
        '_FillValue':fill_value, 
        'missing_value':missing_value, 
        'zlib': compress, 'complevel': complevel # USE COMPRESSION?
    }
print(climate_enc)
# for _var in self.vars:
#     self.dataset[_var].rio.update_encoding(climate_enc, inplace=True)
    
# if  not Path(out_file).exists() or overwrite:
#     self.dataset.to_netcdf(
#             out_file, 
#             # encoding=encoding, 
#             engine="netcdf4",
#             # unlimited_dims={'time':True}
#         )
# else:
#     raise FileExistsError(
#         'The file {out_file} exists and `overwrite` is False'
#     )

{'_FillValue': 1e+20, 'missing_value': 1e+20, 'zlib': True, 'complevel': 9}


In [14]:
climate_enc = {
    '_FillValue':1.e+20, 
    'missing_value':1.e+20, 
    'zlib': True, 'complevel': 9 # USE COMPRESSION?
}

for _var in self.vars:
    self.dataset[_var].rio.update_encoding(climate_enc, inplace=True)
    
if  not Path(out_file).exists() or overwrite:
    self.dataset.to_netcdf(
            out_file, 
            # encoding=encoding, 
            engine="netcdf4",
            # unlimited_dims={'time':True}
        )
else:
    raise FileExistsError('The file {out_file} exists and `overwrite` is False')